# LDpred2 Pipeline for Polygenic Risk Score Prediction

This notebook shows the pipepline for genome-wide PRS prediction using R package `bigsnpr`. 

## Aim

The pipeple was developed to predict PRS using infinitesimal, grid and auto model to estimate effect size. 

## Method

### LDpred

Usually phenotype $Y$ is modeled as a linear combination of $M$ genetic effects, $P$ covaritates and an independent random noise. Idealy, The (marginal) least-squares estimate of an individual marker effect is $\hat\beta_j=X_j^\prime Y/N$

$$
Y = \sum_{i=j}^{M} X_{j} \beta_{j} + \sum_{j=1}^{P} Z_{j} \alpha_{j} + \varepsilon \tag{1}
$$

Ldpred is LDpred is a Bayesian PRS that account for the effects of linkage disequilibrium (LD). It estimates posterior mean causaul effect sizes from GWAS summary statistics by assuming **genetic *architecture* prior** and **LD information from a reference panel**. 



#### LD information from a reference panel (?????)

Correlation among variants can be calculated from reference panel. “LD structure should ideally be similar to the training data for which the summary statistics are calculated.” The reference panel applied in the pipeline is 1000 genomes project (phase 3) data including 2490 (mostly unrelated) individuals and ~1.7M SNPs in common with either HapMap3 or the UK Biobank.

Both 1000 genomes project and hapmap3 has individuals from all populations. The intersection of 1000G and hapmap3 can be general reference panel.

#### Genetic architecture prior

The **prior for effect sizes** is a point-normal mixture distribution, has 2 hyper-parameters: 

* **Heritability (parameter) explained by the genotypes** $h_g^2$

The heritability is estimated from LD score regression and is used as initial parameter for LDpred2.

* **the fraction of causal markers** (i.e., the fraction of markers with non-zero effects) $p$

The distribution of effect size for variant $j$ is given as

$$
\beta_{j}  \sim\left\{\begin{array}{ll}
\mathcal{N}\left(0, \frac{h_g^{2}}{M p}\right) & \text { with probability } \mathrm{p} \\
0 & \text { otherwise }
\end{array}\right. \tag{2}
$$

    (a) LDpred-inf (infinitesimal model)
    
In this case, all markers are **causal** ($p$=1), and effect drawn from a Gaussian distribution, i.e., $\beta_{ij} \sim_{i i d} N\left(0,\left(h_{g}^{2} / M\right)\right)$. The posterior mean can be derived analytically

$$
E(\beta_j \mid \tilde{\beta}_j, D) \approx\left(\frac{M}{N h_{g}^{2}} I+D\right)^{-1} \tilde{\beta}_j \tag{3}
$$

where $\tilde{\beta}_{j}$ denotes a vector of marginally estimated least-squares estimates obtained from the GWAS summary statistics. $D$ denotes the LD matrix between the markers in the training data.
    
    (b) LDpred-grid/auto (non-infinitesimal model)

Without considering LD, the posterior mean of effect size can be derived as 

$$
\mathrm{E}\left(\beta_{j} \mid \tilde{\beta}_{j}\right)=\left(\frac{ h_{g}^{2}}{h_{g}^{2}+\frac{M p}{N}}\right) \bar{p}_{j} \tilde{\beta}_{j} \tag{4}
$$

where $\bar p_j$ is the posterior probability that the $j^{th}$ marker is causal.

However, it is very difficult to derive a analytical expression for the posterior mean under a non-infinitesimal Gaussian mixture prior. Therefore, LDpred approximates it numerically by using an approximate MCMC Gibbs sampler. Once posterior mean effect sizes are estimated, they will be applied to genotype data to obtain **PRSs**. 


### LDpred2

LDpred2 is LDpred 2.0. It can estimate effect size without using validation data to tunning hyper-parameters. Plus, it provides better predictive performance when the causal varients in long-range LD regions and sparse.

LDpred2 algorithm relies on an assumption that 

$$
\operatorname{sd}\left(G_{j}\right) \approx \frac{\operatorname{sd}(Y)}{\operatorname{se}\left(\hat{\gamma}_{j}\right) \sqrt{n}} \tag{5}
$$

where $G_j$ the genotype vector for variant $j$, and $\hat{\gamma}_{j}$ is marginal effect of vairant $j$. For binary traits with logistic model, the approximation is 

$$
\operatorname{sd}\left(G_{j}\right) \approx \frac{2}{\operatorname{se}\left(\hat{\gamma}_{j}\right) \sqrt{n_{\mathrm{eff}}}} \tag{6}
$$

where

$$
n_{\mathrm{eff}}=\frac{4}{1 / n_{\text {case }}+1 / n_{\text {control }}} \tag{7}
$$

To ensure the validity of the assumption, quality control on summary statistics is highly recommanded, like removing variants. Removing criteria would be $SD_{ss} < 0.5\times SD_{val}$ or $SD_{ss} > 0.1 + SD_{val}$ or $SD_{ss} < 0.1$ or $SD_{val} < 0.05$. $SD_{ss}$ is the standard deviations derived from the summary statistics (right-hand side of equation). $SD_{val}$ is the standard deviations of genotypes of individuals in the validation set (training set) (left-hand side).

## Input

1. Reference panel for LD matrix and correlation calculation `(.bed/.bim/.fam)`
    - `--bed_file=path`
2. Reference panel data from bed file `(.rds)`
    - `--ref_file=path`
3. Genotype data `.rds`
    - `--geno_file=path`
4. Summary statistics data `(.rds)`
    - `--summstats_file=path`
5. Sample size for estimate the effect size in summary statistics data
    - `--n_eff=2000000`
   

### Data preparation

* Summary statistics data ($n\times 8$)

Column: "chr", "pos", "rsid", "a1", "a0", "beta", "beta_se", "p"

* refence panel (bigSNP class)

    - genotypes ($n\times \text{# of varients}$):
    
    matrix that contains 0,1,2
    
    - fam ($n \times 6$):
    
    "family.ID", "sample.ID", "paternal.ID", "maternal.ID", "sex", "affection"

    - map ($n\times6$):
    
    "chromosome", "marker.ID", "genetic.dist", "physical.pos", "allele1", "allele2"



## Output

The pipeline save the results from every steps.

* QCplot for quality control
    - `--qc_plot='path/QcPlot.png'`
* LD matrix and correlation matrix.
    - `--ld_out='path/LdOutput.RData'`
* SNP heritability $h^2$
    - `--ldreg_out='path/LdRegOut.RData'`
* Estimated effect size (inf/grid/auto)
    - `xxx_beta ='path/xxxBeta.RData'`
* Predicted PRS (inf/grid/auto) and convergence plot of proportion of causal variants $p$ and heritability $h^2$
    - `xxx_prs = 'path/xxxPrs.RData'`
    - `conv_plot = 'path/ConvPlot.png'`

## General workflow

### Step 1: load data

* Convert reference panal from PLINK format to R format. Read from (bed/bim/fam), it generates `.bk`and `.rds` files. Read in data as `bigSNP` class. 

* Perform SNP matching using `snp_match(sumstats, map)`. Match alleles between summary statistics `sumstats` and SNP information from `map` in `bigSNP` class object.

```
sos run ldpred.ipynb load_data \
    --summstats_file <sumstats.rds> \
    --bed_file  <ref_panal.bed>> \
    --ref_file <ref_panal.rds> \
    --geno_file <new_genotype.rds> \
    --new_geno <NewGeno.RData> \
    --summary_stat <SumStats.RData> \
    --qc_in <QcInput.RData> \
    --n_eff 200000
```

### Step 2: quality control

```
sos run ldpred.ipynb QControl \
    --qc_in <QcInput.RData> \
    --qc_plot <QcPlot.png> \
    --ld_in <LdInput.RData> \
    --sd_out <sd.rds>
```

### Step 3: calculate LD matrix for 22 chromosomes and correlation

Calculate LD correlation using `snp_cor(Gna, size, infos.pos)`
* size: for one SNP, window size around this SNP to compute correlations. Window size of 3 cM is applied in this pipeline which is recommanded by the developer.
* infos.pos: specifying the physical position on a chromosome (in base pairs) of each SNP. 

```
sos run ldpred.ipynb LD_corr \
    --ld_in <LdInput.RData> \
    --ld_out <LdOutput.RData>
```

### Step 4: LD score regression

Perform LD score regression using funciton `snp_ldsc()` and obtain SNP heritability $h^2$ . 

```
sos run ldpred.ipynb LD_reg \
    --ld_out <LdOutput.RData> \
    --ldreg_out <LdRegOut.RData>
```

### Step 5: predict PRS

Three models can be applied to predict PRS which are infinitesimal, grid and auto models. 

* Estimate effect size 

    - Infinitesimal model: `snp_ldpred2_inf(corr, df_beta, h2)`
    - Grid model: `snp_ldpred2_grid(corr, df_beta, grid_param)`
        
        gird_param: hyper parameters $p$ and $h^2$
        
    - Auto model: `snp_ldpred2_auto(corr, df_beta, h2_init, vec_p_init)`
    
        vec_p_init: with 30 different initial values for p. Ranges from 0 to 0.9.
        
* Predict PRS

For grid and auto model, the best combination of p and $h^2$ was selected based on largest t score and 3 times of median absolute deviation of predicted PRS.

Note: xxx stands for inf/grid/auto.

```
sos run ldpred.ipynb xxx \
    --ldreg_out <LdRegOut.RData> \
    --new_geno <NewGeno.RData> \
    --xxx_beta <xxxBeta.RData> \
    --xxx_prs <xxxPrs.RData>

```

### Step 6: predict phenotype

## Example command

```
sos run ldpred.ipynb data_load+QControl+LD_corr+LD_reg+auto \
    --summstats_file sumstats.rds \
    --bed_file  1000G.bed \
    --ref_file 1000G.rds \
    --geno_file new_genotype.rds \
    --new_geno './res-data/NewGeno.RData' \
    --summary_stat './res-data/SumStats.RData' \
    --qc_in './res-data/QcInput.RData' \
    --qc_plot './res-data/QcPlot.png' \
    --conv-plot './res-data/ConvPlot.png' \
    --ld_in './res-data/LdInput.RData' \
    --sd_out './res-data/sd.rds' \
    --ld_out './res-data/LdOutput.RData' \
    --ldreg_out './res-data/LdRegOut.RData' \
    --inf_beta './res-data/InfBeta.RData' \
    --grid_beta './res-data/GridBeta.RData' \
    --auto_beta './res-data/AutoBeta.RData' \
    --inf_prs './res-data/InfPrs.RData' \
    --grid_prs './res-data/GridPrs.RData' \
    --auto_prs './res-data/AutoPrs.RData' \
    --response 1 \
    --n_eff 200000
```


## Reference

1. Publication [Modeling Linkage Disequilibrium
Increases Accuracy of Polygenic Risk Scores](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4596916/pdf/main.pdf)
2. Publication [LDpred2: better, faster, stronger](https://www.biorxiv.org/content/10.1101/2020.04.28.066720v3.full.pdf)
3. R package [bigsnpr](https://cran.rstudio.com/web/packages/bigsnpr/bigsnpr.pdf)
4. [Tutorial](https://privefl.github.io/bigsnpr/articles/LDpred2.html)  for LDpred2

## Command Interface

In [3]:
sos run ldpred.ipynb -h

usage: sos run ldpred.ipynb [workflow_name | -t targets] [options] [workflow_options]
  workflow_name:        Single or combined workflows defined in this script
  targets:              One or more targets to generate
  options:              Single-hyphen sos parameters (see "sos run -h" for details)
  workflow_options:     Double-hyphen workflow-specific parameters

Workflows:
  data_load
  QControl
  LD_corr
  LD_reg
  inf
  grid
  auto
  phenopred

Global Workflow Options:
  --summstats-file VAL (as path, required)
                        path to summary statistics files, genotypes, phenotypes
                        and covaraites data "*.rds"
  --bed-file VAL (as path, required)
                        "*.bed"
  --ref-file VAL (as path, required)
                        "*.rds"
  --geno-file VAL (as path, required)
                        "*.rds"
  --n-eff 2000000 (as int)
                        parameter: tmpdir_ = "tmp-dataset"
  --summary-stat 'path/SumStats.RData'
           

: 1

## Global Parameter Setting

In [ ]:
[global]

### Data preparation

# path to summary statistics files, genotypes, phenotypes and covaraites data
# "*.rds"
parameter: summstats_file =  path
# "*.bed"
parameter: bed_file =  path
# "*.rds"
parameter: ref_file = path
# "*.rds"
parameter: geno_file = path

#parameter: tmpdir_ = "tmp-dataset"
parameter: n_eff = 2000000

# Predict PRS
parameter: summary_stat = 'path/SumStats.RData'
parameter: new_geno = 'path/NewGeno.RData'
parameter: qc_in = 'path/QcInput.RData'
parameter: qc_plot = 'path/QcPlot.png'
parameter: conv_plot = 'path/ConvPlot.png'
parameter: sd_out = 'path/sd.rds'
parameter: ld_in = 'path/LdInput.RData'
parameter: ld_out = 'path/LdOutput.RData'
parameter: ldreg_out = 'path/LdRegOut.RData'
parameter: inf_beta = 'path/InfBeta.RData'
parameter: grid_beta = 'path/GridBeta.RData'
parameter: auto_beta = 'path/AutoBeta.RData'
parameter: inf_prs = 'path/InfPrs.RData'
parameter: grid_prs = 'path/GridPrs.RData'
parameter: auto_prs = 'path/AutoPrs.RData'

# Predict Phenotype


# Binary or continuous phenotype
parameter: response = 1

## Workflow



### Predict PRS 

#### Process bed file

In [ ]:
[data_load_10]

input: bed_file

R: expand=True 
    library(bigsnpr)
    try(snp_readBed("{_input}"))

#### Load reference panel and summary statistics

save `sumstats` as `SumStats.RData`

In [ ]:
[data_load_20]

input: summstats_file
output: summary_stat

R: expand=True
    # Read in the summary statistic file
    sumstats <- readRDS("{_input}") 
    sumstats$n_eff = {n_eff}
    save(sumstats, file = "{_output}")

#### SNP matching and get the CM information from 1000 Genome

In [ ]:
[data_load_30]

input: geno = ref_file, sums = summary_stat
output: qc_in
 
R: expand=True 
    library(bigsnpr)
    load("{_input["sums"]}")
    # now attach the genotype object
    obj.bigSNP <- snp_attach("{_input["geno"]}")
    
    # Assign the genotype to a variable for easier downstream analysis
    genotype <- obj.bigSNP$genotypes
    
    # extract the SNP information from the genotype
    map <- obj.bigSNP$map[-(2:3)]
    names(map) <- c("chr", "pos", "a1", "a0")  
    
    # Rename the data structures
    CHR <- map$chr
    POS <- map$pos   

    # perform SNP matching
    info_snp <- snp_match(sumstats, map)
    
    # get the CM information from 1000 Genome
    # will download the 1000G file to the current directory (".")
    POS2 <- snp_asGeneticPos(CHR, POS, dir = ".")
   
    
    # save data to Rdata file
    save(obj.bigSNP, genotype, map, CHR, POS, info_snp, POS2, file = "{_output}")
    

#### Load genotype data (UK Biobank)

For PRS calculation and phenotype comparison.

In [ ]:
[data_load_40]

input: geno_file
output: new_geno

R: expand=True
    # new_geno = 
    # y = 
    # save(new_geno, y, file = "{_output}")

#### Quality Control

In [1]:
[QControl]

input: qc_in
output: sdout = sd_out, ldin = ld_in, qcplot = qc_plot


R: expand=True
    library(bigsnpr)
    suppressMessages(library(tidyverse))
    load("{_input}")
    NCORES = nb_cores()
    ind.val = 1:nrow(genotype)
    sd <- runonce::save_run(
      sqrt(big_colstats(genotype, ind.val, ncores = NCORES)$var),
      file = "{_output["sdout"]}"
    )

    sd_val <- sd[info_snp$`_NUM_ID_`]

    sd_ss <- with(info_snp, 2 / sqrt(n_eff * beta_se^2))

    is_bad <- sd_ss < (0.5 * sd_val) | 
            sd_ss > (sd_val + 0.1) |  ##### fixme>
            sd_ss < 0.1 | 
            sd_val < 0.05
      
    qplot(sd_val, sd_ss, color = is_bad, alpha = I(0.5)) +
      theme_bigstatsr() +
      coord_equal() +
      scale_color_viridis_d(direction = -1) +
      geom_abline(linetype = 2, color = "red") +
      labs(x = "Standard deviations in the validation set",
           y = "Standard deviations derived from the summary statistics",
           color = "Removed?")
    ggsave("{_output["qcplot"]}")      
      
    n = nrow(info_snp)
    print(paste0(length(which(is_bad=="TRUE")), " over ", n, " was removed in Quality Control."))
           
    info_snp = info_snp[!is_bad, ]
           
    save(obj.bigSNP, genotype, map, CHR, POS, info_snp, POS2, file = "{_output["ldin"]}")

#### Calculate LD

In [ ]:
[LD_corr]
input: ldin = ld_in
output: ld_out

R: expand = True
    library(bigsnpr)
    library(data.table)
    library(bigsparser)
    load("{_input["ldin"]}") 
    source("rcode/LD_corr.R")
    save(info_snp, ld, fam.order, corr, NCORES, genotype, file = "{_output}")

#### Perform LD score regression 

In [ ]:
[LD_reg]

input: ld_out
output: ldreg_out

R: expand=True
    library(bigsnpr)
    suppressMessages(library(tidyverse))
    library(data.table)
    load("{_input}")
    df_beta <- info_snp[,c("beta", "beta_se", "n_eff", "_NUM_ID_")]
    ldsc <- snp_ldsc(ld, 
                    length(ld), 
                    chi2 = (df_beta$beta / df_beta$beta_se)^2,
                    sample_size = df_beta$n_eff, 
                    blocks = NULL)
    h2_est <- ldsc[["h2"]]
    save(h2_est, df_beta, corr, NCORES, info_snp, genotype,file = "{_output}")

#### Get adjusted betas

##### Infinitesimal model

In [ ]:
[inf_10]

input: ldreg_out
output: inf_beta

R: expand=True
    library(bigsnpr)
    library(data.table)
    load("{_input}")
    ## adjusted beta ##
    beta_inf <- snp_ldpred2_inf(corr, df_beta, h2 = h2_est)
    # save data
    save(beta_inf, df_beta, corr, NCORES,info_snp, genotype, file = "{_output}")

##### Grid model

In [ ]:
[grid_10]

input: ldreg_out
output: grid_beta

R: expand=True
    library(bigsnpr)
    library(data.table)
    load("{_input}")
    # Prepare data for grid model
    p_seq <- signif(seq_log(1e-4, 1, length.out = 17), 2)
    h2_seq <- round(h2_est * c(0.7, 1, 1.4), 4)
    grid.param <-
        expand.grid(p = p_seq,
                h2 = h2_seq,
                sparse = c(FALSE, TRUE))
    
    # Get adjusted beta from grid model
    beta_grid <- snp_ldpred2_grid(corr, df_beta, grid.param, ncores = NCORES)
    # save data
    save(beta_grid, df_beta, corr, NCORES,info_snp, genotype,grid.param, file = "{_output}")

##### Auto model

In [ ]:
[auto_10]

input: ldreg_out
output: auto_beta

R: expand=True
    library(bigsnpr)
    library(data.table)
    load("{_input}")
    # Get adjusted beta from the auto model
    multi_auto <- snp_ldpred2_auto(
        corr,
        df_beta,
        h2_init = h2_est,
        vec_p_init = seq_log(1e-4, 0.9, length.out = 30),
        ncores = NCORES
    )
    beta_auto <- sapply(multi_auto, function(auto)
        auto$beta_est)
    # save data
    save(beta_auto, df_beta,multi_auto, corr, NCORES,info_snp, genotype, file = "{_output}")

#### Get PRS

##### Infinitesimal model

In [ ]:
[inf_20]

input: infbeta = inf_beta, newgeno = new_geno
output: inf_prs

R: expand=True
    library(bigsnpr)
    library(data.table)
    load("{_input["infbeta"]}")
    load("{_input["newgeno"]}")

    # calculate PRS for all samples
    ind.test <- 1:nrow(new_geno)
    pred_inf <- big_prodVec(    new_geno,
                                beta_inf,
                                ind.row = ind.test,
                                ind.col = info_snp$`_NUM_ID_`)
    save(pred_inf, file = "{_output}")

##### Grid model

In [5]:
[grid_20]

input: gridbeta = grid_beta, newgeno = new_geno
output: grid_prs

R: expand=True
    library(bigsnpr)
    library(data.table)
    load("{_input["gridbeta"]}")
    load("{_input["newgeno"]}")

    ind.test <- 1:nrow(new_geno)
    pred_grid <- big_prodMat(   new_geno, 
                                beta_grid, 
                                ind.col = info_snp$`_NUM_ID_`)
                                
    grid.params$score <- big_univLinReg(as_FBM(pred_grid), y)$score   
    
    best_grid <- grid.params %>%
         mutate(id = row_number()) %>%
         arrange(desc(score)) %>%
         slice(1) %>%
         pull(id) %>%
         beta_grid[, .]

    pred_grid_best <- big_prodVec(new_geno, best_grid_nosp, ind.col = info_snp$`_NUM_ID_`)

    
    save(pred_grid_best, file = "{_output}")

##### Auto model

In [ ]:
[auto_20]

input: autobeta = auto_beta, newgeno = new_geno
output: autoprs = auto_prs, convplot = conv_plot 

R: expand=True
    library(bigsnpr)
    library(data.table)
    library(ggplot2)
    load("{_input["autobeta"]}")
    load("{_input["newgeno"]}")

    
    ## calculate PRS for all samples
    #ind.test <- 1:nrow(new_geno)
    #pred_auto <-
    #    big_prodMat(new_geno,
    #                beta_auto,
    #                ind.row = ind.test,
    #                ind.col = info_snp$`_NUM_ID_`)
    ## scale the PRS generated from AUTO
    #pred_scaled <- apply(pred_auto, 2, sd)
    #final_beta_auto <-
    #    rowMeans(beta_auto[,
    #                abs(pred_scaled -
    #                    median(pred_scaled)) <
    #                    3 * mad(pred_scaled)])
    #pred_auto <-
    #    big_prodVec(new_geno,
    #        final_beta_auto,
    #        ind.row = ind.test,
    #        ind.col = info_snp$`_NUM_ID_`)
    #        
    #ind = abs(pred_scaled - median(pred_scaled)) < 3 * mad(pred_scaled)
    #ind = which(ind=="TRUE")
                        
    auto <- multi_auto[[ind]]
    plot_grid(
      qplot(y = auto$path_p_est) +
        theme_bigstatsr() +
        geom_hline(yintercept = auto$p_est, col = "blue") +
        scale_y_log10() +
        labs(y = "p"),
      qplot(y = auto$path_h2_est) +
        theme_bigstatsr() +
        geom_hline(yintercept = auto$h2_est, col = "blue") +
        labs(y = "h2"),
      ncol = 1, align = "hv"
    )
    ggsave("{_output["convplot"]}", width = 10, height = 7)
    # save(pred_auto, file = "{_output["autoprs"]}")

### Predict phenotype

In [4]:
[phenopred_10]